## 🔮 Deep Learning Basics

$\text{Reminder:}$

- We have a dataset $(x,y)$ representing the true target function $f_t(x)$ (i.e., $y=f_t(x)$)

- Any machine learning model is equivalent to some hypothesis set represented by mathematical function $f(x;θ)$

- The objective of training is to learn the optimal value of the parameters $θ$ (i.e., find $θ^*$ such that $f_{θ^*}(x)≈f_t(x)$)

- Now use $f_{θ^*}(x)$ on real examples; it's a great approximate of $f_t(x)$

In <font color='Orange'>deep learning</font>, and some machine learning algorithms (which we'll see here), this explictly takes place as follows:

1. <font color='SkyBlue'>Directly define </font> the form of $f(x;θ)$

2. <font color='SkyBlue'> Define (or choose) a loss function </font> $J_m$ that for any $θ$ measures the discripancy between $ŷ_m = f(x_m;θ)$ and $y_m$

  - That is, it measures the discripancy between $f_t(x)$ and our model $f(x_m;θ)$

  - Given a training set we can compute $J(θ)=Σ_{m=1}^{M}J_m(y_m, ŷ_m)$ to get the total loss over it

3. <font color='SkyBlue'> Choose an optimizer </font> and use it to solve the optimization problem $\argmin_θ J(θ)$

- This is the training step

- If <font color='SkyBlue'> minimized properly </font> we get the best $θ$ that makes $f(x;θ) ≈ f_t(x)$ and our model should be quite performant!


# 📊 Logistic Regression

### 1. Define hypothesis f(x;θ)

   $$ ŷ = f(x; \theta) = \sigma(w_0 + w_1x_1 + w_2x_2 + \ldots + w_nx_n) = \sigma(w^T u) $$
   
   - Here, $ \sigma(z) $ is the logistic function, given by $ \sigma(z) = \frac{1}{1 + e^{-z}} $.
   
   - It's just a function that gives the model the ability to output probabilities (e.g., ranges from $0$ to $1$)

   - Classifier is still linear (solving $\sigma(w^T u)=0.5$ yields $w^T u=0$)

In [ ]:
import torch
import numpy as np
from tqdm.notebook import tqdm

def σ(z):
    # TODO: Implement the sigmoid function using the exponential function
    return None

def f(x_train, weights):                        # Assumes x_train is a 2D tensor with shape (n_samples, n_features)
    # Define a column of ones t
    num_rows = x_train.shape[0]
    column_of_ones = torch.ones((num_rows, 1))
    # TODO: Add it to x_train to account for the bias term   (ie, x_train becomes a 2D tensor with shape (n_samples, 1 + n_features))  
    u_train = None
    # Apply matrix multiplication between u_train and weights then pass result by sigmoid
    return σ(u_train @ weights)

### Example to check your implementation and verify understanding
x_train = torch.tensor([[-2.0,  8.00],
                        [ 0.6,  3.5],
                        [ 1.2, -1.5],
                        [-3.0,  2.0]])

weights = torch.tensor([0.1, -2.2, 0.5])            # Bias + two feature weights

# Compute sigmoid output
ŷ = f(x_train, weights)
print(ŷ)

In [ ]:
# TODO: Perform the computation done in the last example to compute ŷ on paper and take attach an image in this cell. 
# That is, steps for matrix multiplication or dot product should be shown.

# To have an idea it may be helpful to check the output of σ(torch.dot(torch.tensor([1.0, -2.0, 8.2]), weights))

### 2. Define loss function

   $$J(\theta) = -\frac{1}{M} \sum_{m=1}^{M} \left[ y_m \log(ŷ_m) + (1 - y_m) \log(1 - ŷ_m) \right] $$

  - Least Squares wouldn't work because difference between label values should be meaningless

  - Suppose $y_m=0$ then $J_m$ is $-\log(1 - ŷ_m)$ which is minimum if $ŷ_m=0$
  - Suppose $y_m=1$ then $J_m$ is $-\log(ŷ_m)$ which is minimum if $ŷ_m=1$

In [ ]:
def J(ŷ, y):
    ϵ = 1e-7            # Small value to avoid log(0)
    # TODO: Something is wrong in the next formula, fix it
    return -torch.mean((y) * torch.log(ŷ + ϵ) + (1-y) * torch.log(1 - ŷ + ϵ))

# Example computation
ŷ = torch.tensor([0.01, 0.99])  # Predicted probabilities
y = torch.tensor([0.0, 1.0])    # True labels for the example
loss_example = J(ŷ, y)
print(f'Example Loss: {loss_example.item():.4f}')

In [ ]:
# TODO: In the example above, why is the loss low? Change the example y and ŷ so its more than 10.

### 3. Optimize the loss over θ for training and plug back in hypothesis for prediction


$$\text{solve} \quad \frac{\partial J}{\partial w} = 0$$

Yields something along the lines of
$$Σ_{m=1}^{M}(\frac{-y_mu_m}{1+e^{-y_m*w^t*u_m}})=0$$

Good luck solving for $w$ (not possible). Thankfully, numerical [optimization exists](https://easyai.tech/wp-content/uploads/2019/01/tiduxiajiang-1.png). 

The simplest algorithm for that is gradient descent which minimizes an arbitrary function $g$ w.r.t $θ$ by applying the following until convergence after choosing the learning rate $α$ and initializing $w$:

$$ \theta_{t+1} = \theta_t - \alpha \frac{\partial g}{\partial θ} $$

- 👀 **Important Observation:** Requires only being able to compute $\frac{\partial g}{\partial θ}$ (no need to equate it to zero)

We were able to compute $\frac{\partial J}{\partial w}$ and hence, we can train the logistic regression model by repeatedly applying:

$$ w_{t+1} = w_t - \alpha \frac{\partial J}{\partial w} $$


In [ ]:
# 3. Optimize the loss over θ for training and plug back in hypothesis for prediction
class LogisticRegression:
    def __init__(self):
        # Initialize parameters
        self.weights = None         # w0, w1, ..., wn
        self.losses = []            # Track loss over time
        
    def fit(self, x_train, y_train, α=0.01, num_epochs=100):
                
        n = x_train.shape[1] + 1                # number of parameters
        # TODO: initialize weights as a 1D tensor of zeros with shape (n,). Should it require gradients?
        self.weights = None
                
        for epoch in tqdm(range(num_epochs), desc="Training Epochs"):  # i.e., number of iterations
            # TODO: Predict for each example using the current model parameters
            y_pred = None
            # Compute the loss
            loss = J(y_pred, y_train)
            
            # 2. Backward pass to compute მJⳆმw
            loss.backward()
            
            # 3. Update weights
            with torch.no_grad():
                # TODO: Perform the weight update
                self.weights -=  None
                
                # Reset the gradient as PyTorch accumulates gradients automatically
                self.weights.grad.zero_()
            
            # TODO: Append the loss to the list of losses. Don't forget using .item() to get the value as a Python float
            None
        
    def predict(self, x_val):
        # TODO: Predict for each example using the model
        y_pred_probs = None
        # Convert probabilities to binary classes by rounding the probabilities
        y_pred = (y_pred_probs > 0.5).float()    
        return   y_pred

### 🍳 Make some classification dataset

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

# Generate synthetic dataset using make_blobs
X, y = make_blobs(n_samples=700, centers=2, n_features=2, cluster_std=2.0, random_state=42)

# Split the dataset into train and test sets
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# TODO: Convert the numpy arrays to tensors
x_train, x_val = None, None
y_train, y_val = None, None

### 💪 Training

In [ ]:
# Fit-Predict Pipeline
lr = LogisticRegression()
lr.fit(x_train, y_train)

In [ ]:
# plot losses from lr.losses
import matplotlib.pyplot as plt

plt.style.use('dark_background')
plt.figure(figsize=(9, 4))
plt.plot(lr.losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.show()

### 🔮 Inference

In [ ]:
y_pred = lr.predict(x_val)

In [ ]:
# Calculate accuracy
accuracy = torch.tensor(y_pred == y_val).float().mean()
print(f'Accuracy: {accuracy.item()*100:.2f}%')